### Libraries Needed

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

### Foursquare Credentials to access surrounding Venures in NYC

In [3]:
CLIENT_ID = 'UZW1S0AYBB2BEUQ2DUQZYX4T3CX014DXSBNGB35U5WEPEDCJ' # your Foursquare ID
CLIENT_SECRET = 'KP5TPXE1GHP0VRDELSYULFKYUR102OGHMX1DGHGMYS13DOVQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 60
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UZW1S0AYBB2BEUQ2DUQZYX4T3CX014DXSBNGB35U5WEPEDCJ
CLIENT_SECRET:KP5TPXE1GHP0VRDELSYULFKYUR102OGHMX1DGHGMYS13DOVQ


# 1. Selecting starting point and searching similar restaurants 

### Now let's find the coordinates for New York City as our starting point

In [4]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


### Now that we have our starting coordinates,we'll send the area for similiar restaurants that be Competition 

#### We want to know where they are in order to avoid them for less competition. So let's check a 16000 meter radius of NYC to cover a good portion of the boroughs

In [5]:
search_query = 'Mexican'
radius = 16000 
print(search_query + ' .... OK!')

Mexican .... OK!


In [6]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=UZW1S0AYBB2BEUQ2DUQZYX4T3CX014DXSBNGB35U5WEPEDCJ&client_secret=KP5TPXE1GHP0VRDELSYULFKYUR102OGHMX1DGHGMYS13DOVQ&ll=40.7127281,-74.0060152&v=20180604&query=Mexican&radius=16000&limit=60'

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d005592deb4950026770e02'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d1c1941735',
      'name': 'Mexican Restaurant',
      'pluralName': 'Mexican Restaurants',
      'primary': True,
      'shortName': 'Mexican'}],
    'hasPerk': False,
    'id': '4bd716bd6798ef3b382c668d',
    'location': {'address': '111 Fulton St',
     'cc': 'US',
     'city': 'New York',
     'country': 'United States',
     'crossStreet': 'at William St',
     'distance': 333,
     'formattedAddress': ['111 Fulton St (at William St)',
      'New York, NY 10038',
      'United States'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.70975353435097,
       'lng': -74.006442636788}],
     'lat': 40.70975353435097,
     'lng': -74.006442636788,
     'postalCode': '10038',
     'state': 'NY'},
    'name': 'Chipotle Mexican Grill',
    'ref

### Now let's sort out the relevant values from the JSON and put it into a dataframe

In [8]:
# assign relevant parts of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1c1941735', 'pluralNam...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bd716bd6798ef3b382c668d,111 Fulton St,...,"[111 Fulton St (at William St), New York, NY 1...","[{'label': 'display', 'lat': 40.70975353435097...",40.709754,-74.006443,NaN,10038,NY,Chipotle Mexican Grill,v-1560302994,NaN
1,"[{'id': '4bf58dd8d48988d1c1941735', 'pluralNam...",276933,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/mad-dog--beans-m...,False,4a15a2e8f964a520a9781fe3,83 Pearl St,...,"[83 Pearl St (at Stone St), New York, NY 10004...","[{'label': 'display', 'lat': 40.7042195983736,...",40.704220,-74.010172,NaN,10004,NY,Mad Dog & Beans Mexican Cantina,v-1560302994,32826442
2,"[{'id': '4bf58dd8d48988d1c1941735', 'pluralNam...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a807ab1f964a52067f51fe3,2 Broadway Frnt 4,...,"[2 Broadway Frnt 4 (btwn Beaver & Stone St), N...","[{'label': 'display', 'lat': 40.704888, 'lng':...",40.704888,-74.012846,NaN,10004,NY,Chipotle Mexican Grill,v-1560302994,NaN
3,"[{'id': '4bf58dd8d48988d153941735', 'pluralNam...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ec27c885c5c3d470deb0904,625 Broadway,...,"[625 Broadway (btwn Bleecker & Houston St), Ne...","[{'label': 'display', 'lat': 40.72609068987567...",40.726091,-73.996533,NaN,10012,NY,Chipotle Mexican Grill,v-1560302994,NaN
4,"[{'id': '4bf58dd8d48988d1c1941735', 'pluralNam...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a4925b7f964a52008ab1fe3,200 Varick St Frnt B,...,[200 Varick St Frnt B (Varick between Houston ...,"[{'label': 'display', 'lat': 40.728274, 'lng':...",40.728274,-74.005063,NaN,10014,NY,Chipotle Mexican Grill,v-1560302994,NaN


In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Chipotle Mexican Grill,Mexican Restaurant,111 Fulton St,US,New York,United States,at William St,333,"[111 Fulton St (at William St), New York, NY 1...","[{'label': 'display', 'lat': 40.70975353435097...",40.709754,-74.006443,NaN,10038,NY,4bd716bd6798ef3b382c668d
1,Mad Dog & Beans Mexican Cantina,Mexican Restaurant,83 Pearl St,US,New York,United States,at Stone St,1010,"[83 Pearl St (at Stone St), New York, NY 10004...","[{'label': 'display', 'lat': 40.7042195983736,...",40.704220,-74.010172,NaN,10004,NY,4a15a2e8f964a520a9781fe3
2,Chipotle Mexican Grill,Mexican Restaurant,2 Broadway Frnt 4,US,New York,United States,btwn Beaver & Stone St,1045,"[2 Broadway Frnt 4 (btwn Beaver & Stone St), N...","[{'label': 'display', 'lat': 40.704888, 'lng':...",40.704888,-74.012846,NaN,10004,NY,4a807ab1f964a52067f51fe3
3,Chipotle Mexican Grill,Burrito Place,625 Broadway,US,New York,United States,btwn Bleecker & Houston St,1688,"[625 Broadway (btwn Bleecker & Houston St), Ne...","[{'label': 'display', 'lat': 40.72609068987567...",40.726091,-73.996533,NaN,10012,NY,4ec27c885c5c3d470deb0904
4,Chipotle Mexican Grill,Mexican Restaurant,200 Varick St Frnt B,US,New York,United States,Varick between Houston & King,1732,[200 Varick St Frnt B (Varick between Houston ...,"[{'label': 'display', 'lat': 40.728274, 'lng':...",40.728274,-74.005063,NaN,10014,NY,4a4925b7f964a52008ab1fe3
5,Dahlia's Mexican Restaurant,Mexican Restaurant,353 Greenwich St,US,New York,United States,NaN,763,"[353 Greenwich St, New York, NY 10013, United ...","[{'label': 'display', 'lat': 40.71875385948834...",40.718754,-74.010337,NaN,10013,NY,51b3b7da498efda7cefb2660
6,Chipotle Mexican Grill,Mexican Restaurant,55 E 8th St Frnt 2,US,New York,United States,E 8th St between Broadway & University,2310,[55 E 8th St Frnt 2 (E 8th St between Broadway...,"[{'label': 'display', 'lat': 40.73116193439805...",40.731162,-73.993437,NaN,10003,NY,4ab3da04f964a520c36e20e3
7,Chipotle Mexican Grill,Mexican Restaurant,405 Avenue of the Americas,US,New York,United States,6th Ave and W 8th Street,2380,[405 Avenue of the Americas (6th Ave and W 8th...,"[{'label': 'display', 'lat': 40.73360550295967...",40.733606,-73.999926,NaN,10014,NY,4adf9c01f964a5200f7c21e3
8,Órale! Mexican Kitchen,Mexican Restaurant,341 Grove St,US,Jersey City,United States,btwn Morgan & Bay St,3227,"[341 Grove St (btwn Morgan & Bay St), Jersey C...","[{'label': 'display', 'lat': 40.72047823780504...",40.720478,-74.042872,Harsimus,07302,NJ,515c3ec7e4b0ee49f92d35a8
9,Chipotle Mexican Grill,Mexican Restaurant,275 Greenwich St,US,New York,United States,btwn Murray & Warren St,495,"[275 Greenwich St (btwn Murray & Warren St), N...","[{'label': 'display', 'lat': 40.71504294817927...",40.715043,-74.011030,NaN,10007,NY,503502d6e4b0bd196609a46e


In [10]:
dataframe_filtered.shape

(50, 16)

### Let's take a look at the established Mexican restaurants in the area

In [13]:
dataframe_filtered.name

0                       Chipotle Mexican Grill
1              Mad Dog & Beans Mexican Cantina
2                       Chipotle Mexican Grill
3                       Chipotle Mexican Grill
4                       Chipotle Mexican Grill
5                  Dahlia's Mexican Restaurant
6                       Chipotle Mexican Grill
7                       Chipotle Mexican Grill
8                       Órale! Mexican Kitchen
9                       Chipotle Mexican Grill
10                      Chipotle Mexican Grill
11                      Chipotle Mexican Grill
12                      Chipotle Mexican Grill
13                      Chipotle Mexican Grill
14                      Chipotle Mexican Grill
15                      Chipotle Mexican Grill
16       Sinigual Contemporary Mexican Cuisine
17                      Chipotle Mexican Grill
18                          QDOBA Mexican Eats
19                      Chipotle Mexican Grill
20                      Chipotle Mexican Grill
21           

### 50 related restaurants with 16000 meters (~10 miles) of NYC!

In [14]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent Downtown New York City
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown NYC',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# adding the Mexican restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

### Already we can see that the largest amount of Mexican restaurants are in the Borough of Manhattan. However the Brooklyn Borough appears to be lacking Mexican resturants in comparison of Manhattan while still maintaining close proximity to the downtown area.

Let's check out West Brooklyn as it isn't overcrowded and our clients would like to stay central

# 2. Let's explore the competition's public ratings in Brooklyn

In [15]:
dataframe_filtered.iloc[40]

name                                       Guadalupana Mexican Bakery
categories                                                     Bakery
address                                                 308 Ellery St
cc                                                                 US
city                                                         Brooklyn
country                                                 United States
crossStreet                       btwn Broadway & Marcus Garvey Blvd.
distance                                                         5758
formattedAddress    [308 Ellery St (btwn Broadway & Marcus Garvey ...
labeledLatLngs      [{'label': 'display', 'lat': 40.69946277391374...
lat                                                           40.6995
lng                                                          -73.9401
neighborhood                                                      NaN
postalCode                                                      11206
state               

In [16]:
dataframe_filtered.iloc[44]

name                                           Chipotle Mexican Grill
categories                                         Mexican Restaurant
address                                               185 Montague St
cc                                                                 US
city                                                         Brooklyn
country                                                 United States
crossStreet                                   Between Court & Clinton
distance                                                         2370
formattedAddress    [185 Montague St (Between Court & Clinton), Br...
labeledLatLngs      [{'label': 'display', 'lat': 40.69434930755160...
lat                                                           40.6943
lng                                                          -73.9918
neighborhood                                                      NaN
postalCode                                                      11201
state               

In [17]:

dataframe_filtered.iloc[13]

name                                           Chipotle Mexican Grill
categories                                         Mexican Restaurant
address                                                  404 Broadway
cc                                                                 US
city                                                         New York
country                                                 United States
crossStreet                                             Walker Street
distance                                                          748
formattedAddress    [404 Broadway (Walker Street), New York, NY 10...
labeledLatLngs      [{'label': 'display', 'lat': 40.7187624898711,...
lat                                                           40.7188
lng                                                          -74.0021
neighborhood                                                      NaN
postalCode                                                      10013
state               

### As 2 out of 3 are fast food chains, we can focus on the Mexican Bakery as it is our direct competition offering home made meals. Let's confirm if Guadalupana is the best of the area by the ratings.

### Send GET request for result & to see the three Brooklyn restaurants' ratings

In [18]:
venue_id = '53dd1f41498efb39049ce081' # ID of Chipotle Mexican Grill - 185 Montague St
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.5


In [19]:
venue_id = '4ad4a7c0f964a52085e820e3' # ID of Chipotle Mexican Grill - 1 Metrotech Center
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.7


In [20]:
venue_id = '4db2354b0cb67b3dafd00977' # ID of Guadalupana Mexican Bakery
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


Not good ratings. Since Guadalupana Mexican Bakery has the better rating, let's explore it further.

### Respectable ratings! 8.4! 

### Let's find out what everyday people think of Guadalupana

In [21]:
# Guadalupana Mexican Bakery Tips
limit = 10 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d0074a389b06a002ce54afc'},
 'response': {'tips': {'count': 3,
   'items': [{'agreeCount': 1,
     'authorInteractionType': 'liked',
     'canonicalUrl': 'https://foursquare.com/item/5af57f773b8307382fa708e9',
     'createdAt': 1526038391,
     'disagreeCount': 0,
     'id': '5af57f773b8307382fa708e9',
     'lang': 'en',
     'lastUpvoteTimestamp': 1553354398,
     'lastVoteText': 'Upvoted Mar 23',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'text': "So good! Real Mexican pastries are amazing! It's also very reasonably priced. Their tres leches cake is to die for.",
     'todo': {'count': 0},
     'type': 'user',
     'user': {'firstName': 'Liz',
      'gender': 'female',
      'id': '6308921',
      'lastName': 'M',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/MAD5FP1N3TMJBQNL.jpg'}}}]}}}

Get a list of features and tips

In [22]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'logView', 'lastUpvoteTimestamp', 'canonicalUrl', 'type', 'text', 'lastVoteText', 'lang', 'user', 'agreeCount', 'likes', 'todo', 'disagreeCount', 'authorInteractionType'])

In [23]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,So good! Real Mexican pastries are amazing! It's also very reasonably priced. Their tres leches cake is to die for.,1,0,5af57f773b8307382fa708e9,Liz,M,female,6308921


Now remember that because I am using a personal developer account, I can access only 2 of the restaurant's tips, instead of all 10 tips.

# 3. MOIA Annual Report 2018 
The top ten countries of origin for foreign-born City residents (NYC Immigration Status)

In [24]:
from pandas import DataFrame
#The top ten countries of origin for foreign-born City residents
ImmStat = {'Country of origin for NYC immigrants': ['Dominican Republic','China','Mexico','Jamaica','Guyana','Ecuador','Haiti','Trinidad & Tobago','Bangladesh','India'],
        'Number': ['422,244','318,581','183,197','176,718','136,760','129,693','88,383','83,222','80,448','76,424'], '% of all foreign-born': ['13.5%','10.2%','5.8%','5.6%','4.4%','4.1%','2.8%','2.7%','2.6%','2.4%']
        }

NYCFor = DataFrame(ImmStat,columns= ['Country of origin for NYC immigrants', 'Number', '% of all foreign-born'])

NYCFor

,Country of origin for NYC immigrants,Number,% of all foreign-born
0,Dominican Republic,"422,244",13.5%
1,China,"318,581",10.2%
2,Mexico,"183,197",5.8%
3,Jamaica,"176,718",5.6%
4,Guyana,"136,760",4.4%
5,Ecuador,"129,693",4.1%
6,Haiti,"88,383",2.8%
7,Trinidad & Tobago,"83,222",2.7%
8,Bangladesh,"80,448",2.6%
9,India,"76,424",2.4%



# 4.  Map of NYC with Respective Neighborhoods Overlaid

In [25]:
import json

!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


In [26]:
neighborhoods_data = newyork_data['features']
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [27]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [28]:

#amap of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# added markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork) 
       
map_newyork